The goal of this file is to vectorize sentences.

In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gensim.models import Word2Vec

In [3]:
# Load the data
train = pd.read_excel('datasets/all_train_data.xlsx')

In [6]:
# Vectorize the data
# Tokenize the data
train['tokenized'] = train['avis_en'].apply(lambda x: str(x).split())

# Create the model
model = Word2Vec(train['tokenized'], min_count=1)

AttributeError: 'float' object has no attribute 'split'

In [4]:
def compute_embedding(text, model):
    return np.mean([model[word] for word in text.split()], axis=0)

# Compute the embedding
train['embedding'] = train['text'].apply(lambda x: compute_embedding(x, model))

KeyError: 'text'